In [2]:
from mlflow.tracking import MlflowClient
MLFLOW_SET_TRACKING_URI = "http://127.0.0.1:5000"
client = MlflowClient(tracking_uri=MLFLOW_SET_TRACKING_URI)

In [5]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1', creation_time=1746683774913, experiment_id='1', last_update_time=1746683774913, lifecycle_stage='active', name='ny-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/0', creation_time=1746683599358, experiment_id='0', last_update_time=1746683599358, lifecycle_stage='active', name='Default', tags={}>]